In [1]:
from imports import *

# 1. Cleaning Democracy Index

Little cleaning needs to be done to the democracy index other than imputing for the 2007 and 2009 value which is missing for every country. This is not a problem and as we discuss in our Milestone 2 write up, we simply just take the average of 2006 and 2008 and 2008 and 2010 since we do not expect the index to fluctuate significantly from year to year. There is no issue with the missing data from 2023 since the WVS does not go to 2023. Since 2005 is missing, we will also just impute 2005 as the 2006 value of the democracy index.

In [2]:
dem_index = pd.read_csv("raw_data/democracy_index.csv")

dem_index.head()

,Entity,Code,Year,Democracy score
0,Afghanistan,AFG,2006,3.06
1,Afghanistan,AFG,2008,3.02
2,Afghanistan,AFG,2010,2.48
3,Afghanistan,AFG,2011,2.48
4,Afghanistan,AFG,2012,2.48


In [3]:
# get number of observations for each year in the dataset
print(dem_index["Year"].value_counts())

# get the number of missing values for each variable - wee see missing in 2007 and 2009
dem_index.isnull().sum()

Year
2006    174
2008    174
2010    174
2011    174
2012    174
2013    174
2014    174
2015    174
2016    174
2017    174
2018    174
2019    174
2020    174
2021    174
2022    174
2023    155
Name: count, dtype: int64


Entity              0
Code               96
Year                0
Democracy score     0
dtype: int64

In [4]:
# we will now impute for 2007 by taking the average of 2006 and 2008

data_2006 = dem_index[dem_index["Year"] == 2006]
data_2008 = dem_index[dem_index["Year"] == 2008]

merged_06_08 = pd.merge(data_2006, data_2008, on=['Entity', 'Code'], suffixes=('_2006', '_2008'))

merged_06_08['Democracy score_2007'] = (merged_06_08['Democracy score_2006'] + merged_06_08['Democracy score_2008']) / 2

data_2007 = merged_06_08[['Entity', 'Code']].copy()
data_2007['Year'] = 2007
data_2007['Democracy score'] = merged_06_08['Democracy score_2007']

dem_index_full = pd.concat([dem_index, data_2007], ignore_index=True)

dem_index_full = dem_index_full.sort_values(by=['Entity', 'Code', 'Year']).reset_index(drop=True)

In [ ]:
# we will create a 2005 row for the countries that have data from 2006 onwards

# ignore warnings
pd.options.mode.chained_assignment = None

data_2005 = dem_index_full[dem_index_full["Year"] == 2006]

data_2005['Year'] = 2005

dem_index_full = pd.concat([dem_index_full, data_2005], ignore_index=True)

dem_index_full = dem_index_full.sort_values(by=['Entity', 'Code', 'Year']).reset_index(drop=True)

In [ ]:
# now we will impute for 2009 by taking the average of 2008 and 2010

data_2008 = dem_index_full[dem_index_full["Year"] == 2008]

data_2010 = dem_index_full[dem_index_full["Year"] == 2010]

merged_08_10 = pd.merge(data_2008, data_2010, on=['Entity', 'Code'], suffixes=('_2008', '_2010'))

merged_08_10['Democracy score_2009'] = (merged_08_10['Democracy score_2008'] + merged_08_10['Democracy score_2010']) / 2

data_2009 = merged_08_10[['Entity', 'Code']].copy()

data_2009['Year'] = 2009

data_2009['Democracy score'] = merged_08_10['Democracy score_2009']

dem_index_full = pd.concat([dem_index_full, data_2009], ignore_index=True)

dem_index_full = dem_index_full.sort_values(by=['Entity', 'Code', 'Year']).reset_index(drop=True)

dem_index_full.head()

,Entity,Code,Year,Democracy score
0,Afghanistan,AFG,2005,3.06
1,Afghanistan,AFG,2006,3.06
2,Afghanistan,AFG,2007,3.04
3,Afghanistan,AFG,2008,3.02
4,Afghanistan,AFG,2009,2.75
...,...,...,...,...
3282,Zimbabwe,ZWE,2019,3.16
3283,Zimbabwe,ZWE,2020,3.16
3284,Zimbabwe,ZWE,2021,2.92
3285,Zimbabwe,ZWE,2022,2.92


# 2. Cleaning World GDP Data

The World GDP data has little to no missing data for countries that are in the WVS, though we have to rename some countries to merge it into the democracy index. We also have a few countries with missing data so we supplement it with IMF data that is measured in the same way (current dollars). 

In [7]:
# load in world_gdp_data.csv without the first row and second row make the second row the header

world_gdp = pd.read_csv("raw_data/world_gdp_data.csv", skiprows=3)

# remove all columns before 2005 but excluding country name and country code

world_gdp = world_gdp.drop(world_gdp.columns[2:world_gdp.columns.get_loc('2005')], axis=1)

# remove Unnamed: 68 column

world_gdp = world_gdp.drop('Unnamed: 68', axis=1)

# turn the dataframe into long format

world_gdp_long = world_gdp.melt(id_vars=['Country Name', 'Country Code'], var_name='year', value_name='gdp_per_capita')

world_gdp_long['year'] = world_gdp_long['year'].astype(int)

# rename countries as needed to merge onto dem index

world_gdp_long['Country Name'] = world_gdp_long['Country Name'].replace({
    'Egypt, Arab Rep.': 'Egypt',
    'Hong Kong SAR, China': 'Hong Kong',
    'Iran, Islamic Rep.': 'Iran',
    'Korea, Rep.': 'South Korea',
    'Kyrgyz Republic': 'Kyrgyzstan',
    'Russian Federation': 'Russia',
    'Turkiye': 'Turkey',
    'Slovak Republic': 'Slovakia',
    'Syrian Arab Republic': 'Syria',
    'Venezuela, RB': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'Yemen, Rep.': 'Yemen'
})


world_gdp_long.head()

,Country Name,Country Code,year,gdp_per_capita
0,Aruba,ABW,2005,24978.221674
1,Africa Eastern and Southern,AFE,2005,1130.168938
2,Afghanistan,AFG,2005,254.115276
3,Africa Western and Central,AFW,2005,1008.341766
4,Angola,AGO,2005,1900.723809


In [8]:
# left join dem_index_full with world_gdp_long 

dem_index_gdp = pd.merge(dem_index_full, world_gdp_long, left_on=['Entity', 'Year'], right_on=['Country Name', 'year'], how='left')



In [9]:
# read in gdp data supplement

gdp_data_supplement = pd.read_csv("raw_data/gdp_data_supplement.csv")

# transform the data into long format

gdp_data_supplement_long = gdp_data_supplement.melt(id_vars=['country'], var_name='year', value_name='gdp_per_capita')

gdp_data_supplement_long['year'] = gdp_data_supplement_long['year'].astype(int)

# rename Taiwan Province of China to Taiwan, West Bank and Gaza to Palestine

gdp_data_supplement_long['country'] = gdp_data_supplement_long['country'].replace({
    'Taiwan Province of China': 'Taiwan',
    'West Bank and Gaza': 'Palestine'})



# merge gdp_data_supplement_long onto dem_index_gdp to get full data set

dem_index_gdp = pd.merge(dem_index_gdp, gdp_data_supplement_long, left_on=['Entity', 'Year'], right_on=['country', 'year'], how='left')

# set gdp_per_capita_x to gdp_per_capita_y if gdp_per_capita_x is missing

dem_index_gdp['gdp_per_capita_x'] = dem_index_gdp['gdp_per_capita_x'].fillna(dem_index_gdp['gdp_per_capita_y'])

dem_index_gdp = dem_index_gdp.drop('gdp_per_capita_y', axis=1)

dem_index_gdp = dem_index_gdp.rename(columns={'gdp_per_capita_x': 'gdp_per_capita'})

# remove year_y and country and year_x and Country Name and Country Code 

dem_index_gdp = dem_index_gdp.drop(['year_y', 'country', 'year_x', 'Country Name', 'Country Code'], axis=1)

# remove any code that is missing

dem_index_gdp = dem_index_gdp.dropna(subset=['Code'])


# remove Bhutan, Cape Verde, Congo, Democratic Republic of Congo, Cuba, East Timor, Eritrea, Gambia, Laos, North Korea, Syria 

dem_index_gdp = dem_index_gdp[~dem_index_gdp['Entity'].isin(['Bhutan', 'Cape Verde', 'Congo', 'Democratic Republic of Congo', 'Cuba', 'East Timor', 'Eritrea', 'Gambia', 'Laos', 'North Korea', 'Syria'])]




In [10]:
# standardize dem_index_gdp 

dem_index_gdp['gdp_per_capita_std'] = (dem_index_gdp['gdp_per_capita'] - dem_index_gdp['gdp_per_capita'].mean()) / dem_index_gdp['gdp_per_capita'].std()
    

# 3. Human Development Index Data

There is data on every country we care about in the WVS except for Taiwan likely because of conflicts in the UN regarding China not recognizing Taiwan as a state. We thus remove Taiwan from our analysis.

In [11]:
# read in hdi 

hdi = pd.read_csv("raw_data/Human_Development_Index_and_Adjusted.csv", encoding='ISO-8859-1')

hdi.head()

# keep first 50 columns and remove the rest

hdi = hdi.iloc[:, :50]

# remove hdicode, region, hdi_rank_2022
hdi = hdi.drop(['hdicode', 'region', 'hdi_rank_2022'], axis=1)

# remove hdi_ from column names 

hdi.columns = hdi.columns.str.replace('hdi_', '')

hdi

# remove all columns with le_ prefix

hdi = hdi.loc[:, ~hdi.columns.str.startswith('le_')]

# make the data long 

hdi_long = hdi.melt(id_vars=['country', 'iso3'], var_name='year', value_name='hdi')

hdi_long


hdi_long['year'] = hdi_long['year'].astype(int)

# remove if year is less than 2005

hdi_long = hdi_long[hdi_long['year'] >= 2005]

# rename countries as needed to merge onto dem index

# rename Bolivia (Plurinational State of) to Bolivia

hdi_long['country'] = hdi_long['country'].replace({

    'Bolivia (Plurinational State of)': 'Bolivia',
    'Hong Kong, China (SAR)': 'Hong Kong',
    'Moldova (Republic of)': 'Moldova',
    'Iran (Islamic Republic of)': 'Iran',
    'Palestine, State of': 'Palestine', 
    'Russian Federation': 'Russia',
    'Korea (Republic of)': 'South Korea',
    'Tanzania (United Republic of)': 'Tanzania',
    'Türkiye': 'Turkey',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam'
})


In [12]:
# merge hdi_long onto dem_index_gdp using year and country 

dem_index_gdp_hdi = pd.merge(dem_index_gdp, hdi_long, left_on=['Entity', 'Year'], right_on=['country', 'year'], how='left')

In [13]:
# drop if Year = 2023

dem_index_gdp_hdi = dem_index_gdp_hdi[dem_index_gdp_hdi['Year'] != 2023]

# remove row print limit 

# look at rows with missing hdi

# remove Cote d'Ivoire, Eswatini from the data

dem_index_gdp_hdi = dem_index_gdp_hdi[~dem_index_gdp_hdi['Entity'].isin(["Cote d'Ivoire", 'Eswatini'])]



In [16]:
# add a 2023 row for India which is just a copy of 2022

india_2023 = dem_index_gdp_hdi[dem_index_gdp_hdi['Entity'] == 'India'].copy()

# keep only 2022 row
india_2023 = india_2023[india_2023['Year'] == 2022]

india_2023['Year'] = 2023

dem_index_gdp_hdi = pd.concat([dem_index_gdp_hdi, india_2023], ignore_index=True)

# rename Entity Hong Kong to Hong Kong SAR
dem_index_gdp_hdi['Entity'] = dem_index_gdp_hdi['Entity'].replace({'Hong Kong': 'Hong Kong SAR'})

# Rename United Kingdom to Great Britain
dem_index_gdp_hdi['Entity'] = dem_index_gdp_hdi['Entity'].replace({'United Kingdom': 'Great Britain'})

# Rename United States to United States of America
dem_index_gdp_hdi['Entity'] = dem_index_gdp_hdi['Entity'].replace({'United States': 'United States of America'})


In [17]:
# save dem_index_gdp_hdi to csv

dem_index_gdp_hdi.to_csv("cleaned_data/controls_data.csv", index=False)

The only countries in the WVS not in the controls are Macau, Puerto Rico, and Andorra which we will have to drop from our WVS data.